# 오답 중에 분산 높은 친구들 찾기

In [1]:
import tensorflow as tf
import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

new_x_train = x_train
new_x_test = x_test

_mean = np.mean(x_train)
_std = np.std(x_train)
print('Original image')
print(_mean, _std)
new_x_train = (new_x_train - _mean)/_std
new_x_test = (new_x_test - _mean)/_std
_mean = np.mean(new_x_train)
_std = np.std(new_x_train)
print('Standardization image')
print(_mean, _std)

model = tf.keras.models.load_model('MyCNN_ver01')
model.summary()

Num GPUs Available:  1
Original image
120.70756512369792 64.1500758911213
Standardization image
-2.5247951877342226e-17 1.0000000000000022


2023-01-06 01:01:48.992370: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9970 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                    

### first, find not correct answer index

In [158]:
_ans_var = np.ones([50000, 2])
_result = model.predict(new_x_train)

for i in tqdm.tqdm(range(50000)):
    if (tf.not_equal(tf.argmax(_result[i]), y_train[i])):
        _ans_var[i, 0] = 0
    _ans_var[i, 1] = tf.math.reduce_variance(_result[i])

1563/1563 [==============================] - 4s 2ms/step


100%|██████████| 50000/50000 [00:37<00:00, 1321.12it/s]


### index 포함하는 정보 정렬

In [169]:
np.save('./data/_ans_var_index.npy', _ans_var)

### 분산이 아주 작을 경우에 오답률이 높았던 것으로 확인됨

- 분산이 작으면 헷갈렸다.
- 분산이 크면 확신했다.

In [168]:
_ans_var_sorted = np.sort(_ans_var, axis=0)
for i in range(50000):
        #print(_ans_var[i])
        print(_ans_var_sorted[i])
np.save('./data/_ans_var_sorted.npy', _ans_var_sorted)

[0.        0.0011028]
[0.         0.00131738]
[0.         0.00158002]
[0.         0.00164864]
[0.         0.00168615]
[0.        0.0019442]
[0.         0.00216976]
[0.         0.00226029]
[0.        0.0023972]
[0.         0.00239844]
[0.         0.00244663]
[0.         0.00253839]
[0.         0.00258779]
[0.         0.00260065]
[0.         0.00274568]
[0.         0.00275376]
[0.         0.00275674]
[0.         0.00279735]
[0.         0.00297093]
[0.         0.00301663]
[0.         0.00308014]
[0.         0.00310318]
[0.         0.00315411]
[0.         0.00317156]
[0.         0.00317994]
[0.         0.00320837]
[0.         0.00334469]
[0.         0.00335852]
[0.         0.00336923]
[0.         0.00338443]
[0.         0.00344386]
[0.         0.00344687]
[0.         0.00349316]
[0.         0.00353557]
[0.         0.00355131]
[0.         0.00357472]
[0.         0.00360097]
[0.         0.00360798]
[0.         0.00365932]
[0.         0.00366834]
[0.         0.00368918]
[0.         0.00369544

### 오답에 대해 augmentation 진행하기

1. 동일한 분산이 등장하지 않았으니, [_ans_var_sorted] 변수에서 가장 작은 몇 가지 값을 가지는 index를 찾을 수 있음.
2. 해당 사진들에 대해 augmentation 진행

### 다시 학습시키기